In [48]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from keras.callbacks import EarlyStopping
import math
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

Using matplotlib backend: Qt5Agg


In [49]:
path="D:/scrapping/magicbricks/"
df=pd.read_csv(f'{path}data_gnr_jan.csv')

In [120]:
df.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
Society,Golfforeste AC Apartments,SKA Metro Ville,Prideville,Purvanchal Royal City,Stellar One
addressLocality,Zeta 1,"Eta 2, Eta",Yamuna Expressway,Chi 5,Greater Noida West
agentCompanyName,BD Green Home Pvt. Ltd.,NaN,Imperia Structures Ltd.,Purvanchal Projects Pvt. Ltd.,Stellar Ventures Pvt. Ltd.
agentMaskedmobilenumber,+91-88XXXXXXX90,+91-91XXXXXXX15,+91-74XXXXXXX93,+91-78XXXXXXX42,+91-98XXXXXXX77
agentName,Praveen Choudhary,SKA GROUP,Vipin,Shashi Kapoor,Stellar Ventures Pvt. Ltd.
bathroom,2,2,2,4,2
bedroom,2,2,2,4,2
builderName,Paramount Group,SKA Group,Imperia Structures Ltd.,Purvanchal Projects Pvt. Ltd.,Stellar Ventures Pvt. Ltd.
cityName,Greater Noida,Greater Noida,Greater Noida,Greater Noida,Greater Noida


In [51]:
df.floorSize.unique()

array(['1260 FTK ', '920 FTK ', '900 FTK ', '2505 FTK ', '895 FTK ',
       '1380 FTK ', '1730 FTK ', '550 FTK ', '1500 FTK ', '530 FTK ',
       '300 FTK ', '1155 FTK ', '1197 FTK ', '127 FTK ', '133 FTK ',
       '1650 FTK ', '982 FTK ', '1600 FTK ', '1170 FTK ', '2550 FTK ',
       '1060 FTK ', '1075 FTK ', '1165 FTK ', '1970 FTK ', '3710 FTK ',
       nan, '2630 FTK ', '1065 FTK ', '6300 FTK ', '1930 FTK ',
       '3008 FTK ', '1400 FTK ', '500 FTK ', '1475 FTK ', '1350 FTK ',
       '150 FTK ', '3500 FTK ', '1050 FTK ', '1440 FTK ', '1460 FTK ',
       '1227 FTK ', '2350 FTK ', '1150 FTK ', '1000 FTK ', '1563 FTK ',
       '108 FTK ', '955 FTK ', '90 FTK ', '1800 FTK ', '1900 FTK ',
       '1450 FTK ', '1458 FTK ', '1375 FTK ', '925 FTK ', '2900 FTK ',
       '2490 FTK ', '690 FTK ', '1230 FTK ', '890 FTK ', '799 FTK ',
       '1740 FTK ', '1485 FTK ', '1700 FTK ', '1245 FTK ', '58 FTK ',
       '3876 FTK ', '1286 FTK ', '1040 FTK ', '1114 FTK ', '2495 FTK ',
       '1425 FTK ', '

In [52]:
#df['price/sqft']=df['price']/df['floorSize']

In [53]:
#df['floorSize'].min()

In [54]:
df.columns

Index(['Unnamed: 0', 'Society', 'addressLocality', 'agentCompanyName',
       'agentMaskedmobilenumber', 'agentName', 'bathroom', 'bedroom',
       'builderName', 'cityName', 'createdDate', 'description', 'floorSize',
       'floorno', 'furnshingstatus', 'id', 'latitude', 'locality', 'longitude',
       'name', 'numberOfRooms', 'postedBy', 'price', 'priceInWord',
       'projectName', 'propertyType', 'url'],
      dtype='object')

In [55]:
df.description.unique()[0]

'2 Bath,Unfurnished,16 floor Paramount Golf Forest Ac Apartments complimented by designer landscape with a green golf course for you and your family, a well-deserved healthy lifestyle.'

In [56]:
df.name.unique()[0]

'2 BHK Apartment for Sale in Golfforeste AC Apartments, Zeta 1'

In [57]:
df.locality[0]

'Zeta 1'

In [58]:
df.locality.nunique()

106

In [59]:
df.shape

(1555, 27)

In [60]:
df['createdDate']=pd.to_datetime(df['createdDate'])

In [61]:
df['createdDate']=df['createdDate'].dt.strftime('%m/%d/%Y')

In [62]:
def null_values(DataFrame_Name):
    
    sum_null = DataFrame_Name.isnull().sum()
    total_count = DataFrame_Name.isnull().count()
    percent_nullvalues = sum_null/total_count * 100
    df_null = pd.DataFrame()
    df_null['Total_values'] = total_count
    df_null['Null_Count'] = sum_null
    df_null['Percent'] = percent_nullvalues
    df_null = df_null.sort_values(by='Null_Count',ascending = False)

    return(df_null)

In [63]:
df=df[~df.furnshingstatus.isnull()]

In [64]:
null_values(df)

,Total_values,Null_Count,Percent
agentCompanyName,1549,1341,86.571982
builderName,1549,801,51.710781
Society,1549,789,50.936088
projectName,1549,789,50.936088
latitude,1549,650,41.962556
longitude,1549,650,41.962556
floorno,1549,163,10.522918
locality,1549,94,6.068431
addressLocality,1549,94,6.068431
floorSize,1549,45,2.905100


In [65]:
df[df['furnshingstatus'].isnull()]

,Unnamed: 0,Society,addressLocality,agentCompanyName,agentMaskedmobilenumber,agentName,bathroom,bedroom,builderName,cityName,...,locality,longitude,name,numberOfRooms,postedBy,price,priceInWord,projectName,propertyType,url


In [66]:
df.projectName.fillna('Not available',inplace=True)

In [67]:
df.floorno.fillna('Not available',inplace=True)

In [68]:
df.price.fillna('Not define',inplace=True)

In [69]:
def replace_ftk(x):
    #print(x)
    if(re.findall(r'FTK',x)):
        result = re.sub(r'FTK','',x)
    else:
        result = x
    return (result)

In [70]:
def replace_int(x):
    #print(x)
    if x !='nan':
        if(re.findall("[$&+,:;=?@#|'<>.^*()%!-]",x)):
            result = re.sub("[$&+,:;=?@#|'<>.^*()%!-]",'',x)
        else:
            result = x
    else:
        result =0
    return result

In [71]:
df.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
Society,Golfforeste AC Apartments,SKA Metro Ville,Prideville,Purvanchal Royal City,Stellar One
addressLocality,Zeta 1,"Eta 2, Eta",Yamuna Expressway,Chi 5,Greater Noida West
agentCompanyName,BD Green Home Pvt. Ltd.,NaN,Imperia Structures Ltd.,Purvanchal Projects Pvt. Ltd.,Stellar Ventures Pvt. Ltd.
agentMaskedmobilenumber,+91-88XXXXXXX90,+91-91XXXXXXX15,+91-74XXXXXXX93,+91-78XXXXXXX42,+91-98XXXXXXX77
agentName,Praveen Choudhary,SKA GROUP,Vipin,Shashi Kapoor,Stellar Ventures Pvt. Ltd.
bathroom,2,2,2,4,2
bedroom,2,2,2,4,2
builderName,Paramount Group,SKA Group,Imperia Structures Ltd.,Purvanchal Projects Pvt. Ltd.,Stellar Ventures Pvt. Ltd.
cityName,Greater Noida,Greater Noida,Greater Noida,Greater Noida,Greater Noida


In [72]:
df['floorSize']=df.floorSize.apply(lambda x : replace_ftk(str(x)))

In [73]:
df_recommend=df[['Society', 'locality', 'bathroom', 'bedroom',
       'builderName', 'cityName','floorSize',
       'floorno', 'furnshingstatus','name','price', 'priceInWord', 'propertyType','description']]

In [74]:
df_recommend.head()

,Society,locality,bathroom,bedroom,builderName,cityName,floorSize,floorno,furnshingstatus,name,price,priceInWord,propertyType,description
0,Golfforeste AC Apartments,Zeta 1,2.0,2.0,Paramount Group,Greater Noida,1260,16,Unfurnished,2 BHK Apartment for Sale in Golfforeste AC Apa...,3.71e+06,37.1 Lac,Apartment,"2 Bath,Unfurnished,16 floor Paramount Golf For..."
1,SKA Metro Ville,"Eta 2, Eta",2.0,2.0,SKA Group,Greater Noida,920,Not available,Semi-Furnished,"2 BHK Apartment for Sale in SKA Metro Ville, E...",2.56e+06,25.6 Lac,Apartment,"2 Bath,Semi-Furnished SKA Metro Ville is the f..."
2,Prideville,Yamuna Expressway,2.0,2.0,Imperia Structures Ltd.,Greater Noida,900,Not available,Semi-Furnished,"2 BHK Apartment for Sale in Prideville , Yamun...",4.68e+06,46.8 Lac,Apartment,"2 Bath,Semi-Furnished We at Prideville underst..."
3,Purvanchal Royal City,Chi 5,4.0,4.0,Purvanchal Projects Pvt. Ltd.,Greater Noida,2505,19,Semi-Furnished,4 BHK Apartment for Sale in Purvanchal Royal C...,1.12e+07,1.12 Cr,Apartment,"4 Bath,Semi-Furnished,19 floor A life where yo..."
4,Stellar One,Greater Noida West,2.0,2.0,Stellar Ventures Pvt. Ltd.,Greater Noida,895,Not available,Unfurnished,"2 BHK Apartment for Sale in Stellar One, Great...",3.11e+06,31.1 Lac,Apartment,"2 Bath,Unfurnished In Greater Noida(West), alo..."


In [75]:
null_values(df_recommend)

,Total_values,Null_Count,Percent
builderName,1549,801,51.710781
Society,1549,789,50.936088
locality,1549,94,6.068431
bedroom,1549,33,2.130407
bathroom,1549,2,0.129116
cityName,1549,0,0.000000
floorSize,1549,0,0.000000
floorno,1549,0,0.000000
furnshingstatus,1549,0,0.000000
name,1549,0,0.000000


In [76]:
#df_recommend[df_recommend.Society.isnull()]

In [77]:
df_recommend.columns

Index(['Society', 'locality', 'bathroom', 'bedroom', 'builderName', 'cityName',
       'floorSize', 'floorno', 'furnshingstatus', 'name', 'price',
       'priceInWord', 'propertyType', 'description'],
      dtype='object')

In [78]:
col_list=['Society', 'locality', 'bathroom', 'bedroom', 'builderName', 'cityName',
       'floorSize', 'floorno', 'furnshingstatus', 'name', 'price',
       'priceInWord', 'propertyType','description']

In [79]:
df_recommend.head()

,Society,locality,bathroom,bedroom,builderName,cityName,floorSize,floorno,furnshingstatus,name,price,priceInWord,propertyType,description
0,Golfforeste AC Apartments,Zeta 1,2.0,2.0,Paramount Group,Greater Noida,1260,16,Unfurnished,2 BHK Apartment for Sale in Golfforeste AC Apa...,3.71e+06,37.1 Lac,Apartment,"2 Bath,Unfurnished,16 floor Paramount Golf For..."
1,SKA Metro Ville,"Eta 2, Eta",2.0,2.0,SKA Group,Greater Noida,920,Not available,Semi-Furnished,"2 BHK Apartment for Sale in SKA Metro Ville, E...",2.56e+06,25.6 Lac,Apartment,"2 Bath,Semi-Furnished SKA Metro Ville is the f..."
2,Prideville,Yamuna Expressway,2.0,2.0,Imperia Structures Ltd.,Greater Noida,900,Not available,Semi-Furnished,"2 BHK Apartment for Sale in Prideville , Yamun...",4.68e+06,46.8 Lac,Apartment,"2 Bath,Semi-Furnished We at Prideville underst..."
3,Purvanchal Royal City,Chi 5,4.0,4.0,Purvanchal Projects Pvt. Ltd.,Greater Noida,2505,19,Semi-Furnished,4 BHK Apartment for Sale in Purvanchal Royal C...,1.12e+07,1.12 Cr,Apartment,"4 Bath,Semi-Furnished,19 floor A life where yo..."
4,Stellar One,Greater Noida West,2.0,2.0,Stellar Ventures Pvt. Ltd.,Greater Noida,895,Not available,Unfurnished,"2 BHK Apartment for Sale in Stellar One, Great...",3.11e+06,31.1 Lac,Apartment,"2 Bath,Unfurnished In Greater Noida(West), alo..."


In [80]:
df_recommend['description']=df_recommend['description'].str.cat(df['name'].astype(str), sep =" , ")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [81]:
df_recommend.head()

,Society,locality,bathroom,bedroom,builderName,cityName,floorSize,floorno,furnshingstatus,name,price,priceInWord,propertyType,description
0,Golfforeste AC Apartments,Zeta 1,2.0,2.0,Paramount Group,Greater Noida,1260,16,Unfurnished,2 BHK Apartment for Sale in Golfforeste AC Apa...,3.71e+06,37.1 Lac,Apartment,"2 Bath,Unfurnished,16 floor Paramount Golf For..."
1,SKA Metro Ville,"Eta 2, Eta",2.0,2.0,SKA Group,Greater Noida,920,Not available,Semi-Furnished,"2 BHK Apartment for Sale in SKA Metro Ville, E...",2.56e+06,25.6 Lac,Apartment,"2 Bath,Semi-Furnished SKA Metro Ville is the f..."
2,Prideville,Yamuna Expressway,2.0,2.0,Imperia Structures Ltd.,Greater Noida,900,Not available,Semi-Furnished,"2 BHK Apartment for Sale in Prideville , Yamun...",4.68e+06,46.8 Lac,Apartment,"2 Bath,Semi-Furnished We at Prideville underst..."
3,Purvanchal Royal City,Chi 5,4.0,4.0,Purvanchal Projects Pvt. Ltd.,Greater Noida,2505,19,Semi-Furnished,4 BHK Apartment for Sale in Purvanchal Royal C...,1.12e+07,1.12 Cr,Apartment,"4 Bath,Semi-Furnished,19 floor A life where yo..."
4,Stellar One,Greater Noida West,2.0,2.0,Stellar Ventures Pvt. Ltd.,Greater Noida,895,Not available,Unfurnished,"2 BHK Apartment for Sale in Stellar One, Great...",3.11e+06,31.1 Lac,Apartment,"2 Bath,Unfurnished In Greater Noida(West), alo..."


In [82]:
df_recommend['floorSize']=df_recommend.floorSize.apply(lambda x : replace_int(str(x)))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [96]:
df_recommend['price']=df_recommend.price.apply(lambda x : replace_int(str(x)))

In [83]:
df_recommend['floorSize']=df_recommend.floorSize.astype(float)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [84]:
df_recommend=df_recommend[df_recommend.floorSize>100]

In [85]:
df_recommend.head()

,Society,locality,bathroom,bedroom,builderName,cityName,floorSize,floorno,furnshingstatus,name,price,priceInWord,propertyType,description
0,Golfforeste AC Apartments,Zeta 1,2.0,2.0,Paramount Group,Greater Noida,1260.0,16,Unfurnished,2 BHK Apartment for Sale in Golfforeste AC Apa...,3.71e+06,37.1 Lac,Apartment,"2 Bath,Unfurnished,16 floor Paramount Golf For..."
1,SKA Metro Ville,"Eta 2, Eta",2.0,2.0,SKA Group,Greater Noida,920.0,Not available,Semi-Furnished,"2 BHK Apartment for Sale in SKA Metro Ville, E...",2.56e+06,25.6 Lac,Apartment,"2 Bath,Semi-Furnished SKA Metro Ville is the f..."
2,Prideville,Yamuna Expressway,2.0,2.0,Imperia Structures Ltd.,Greater Noida,900.0,Not available,Semi-Furnished,"2 BHK Apartment for Sale in Prideville , Yamun...",4.68e+06,46.8 Lac,Apartment,"2 Bath,Semi-Furnished We at Prideville underst..."
3,Purvanchal Royal City,Chi 5,4.0,4.0,Purvanchal Projects Pvt. Ltd.,Greater Noida,2505.0,19,Semi-Furnished,4 BHK Apartment for Sale in Purvanchal Royal C...,1.12e+07,1.12 Cr,Apartment,"4 Bath,Semi-Furnished,19 floor A life where yo..."
4,Stellar One,Greater Noida West,2.0,2.0,Stellar Ventures Pvt. Ltd.,Greater Noida,895.0,Not available,Unfurnished,"2 BHK Apartment for Sale in Stellar One, Great...",3.11e+06,31.1 Lac,Apartment,"2 Bath,Unfurnished In Greater Noida(West), alo..."


In [93]:
df_recommend=df_recommend[df_recommend.price!='Not define']

In [99]:
df_recommend['price']=df_recommend.price.astype(float)

In [100]:
df_recommend['price/area']=df_recommend['price']/df_recommend['floorSize']

In [103]:
df_recommend[df_recommend.locality=='Noida Extension']

,Society,locality,bathroom,bedroom,builderName,cityName,floorSize,floorno,furnshingstatus,name,price,priceInWord,propertyType,description,price/area
77,Shri Radha Sky Garden,Noida Extension,2.0,2.0,Shri Group,Greater Noida,1286.0,8,Unfurnished,2 BHK Apartment for Sale in Shri Radha Sky Gar...,48000000.0,48 Lac,Apartment,"2 Bath,Unfurnished,8 floor This Multistorey A...",37325.038880
86,NaN,Noida Extension,3.0,3.0,NaN,Greater Noida,1852.0,1,Unfurnished,3 BHK Apartment for Sale in Noida Extension,62000000.0,62 Lac,Apartment,"3 Bath,Unfurnished,1 floor,North - East facing...",33477.321814
133,Hawelia Valenova Park,Noida Extension,3.0,3.0,Hawelia Group,Greater Noida,1470.0,7,Furnished,3 BHK Apartment for Sale in Hawelia Valenova P...,51450000.0,51.5 Lac,Apartment,"3 Bath,Furnished,7 floor,North - East facing 8...",35000.000000
170,Nirala Estate,Noida Extension,2.0,2.0,Nirala India,Greater Noida,955.0,10,Unfurnished,"2 BHK Apartment for Sale in Nirala Estate, Noi...",33750000.0,33.8 Lac,Apartment,"2 Bath,Unfurnished,10 floor,East facing Availa...",35340.314136
178,JKG Palm Court,Noida Extension,2.0,2.0,JKG Group,Greater Noida,1090.0,21,Semi-Furnished,"2 BHK Apartment for Sale in JKG Palm Court, No...",34880000.0,34.9 Lac,Apartment,"2 Bath,Semi-Furnished,21 floor,East facing Par...",32000.000000
190,Supertech Eco Village 1,Noida Extension,2.0,2.0,Supertech Ltd.,Greater Noida,890.0,7,Furnished,2 BHK Apartment for Sale in Supertech Eco Vill...,29000000.0,29 Lac,Apartment,"2 Bath,Furnished,7 floor,North - East facing E...",32584.269663
197,Lotus Villas,Noida Extension,1.0,1.0,Renowned Group,Greater Noida,650.0,Ground,Semi-Furnished,"1 BHK Apartment for Sale in Lotus Villas, Noid...",18100000.0,18.1 Lac,Apartment,"1 Bath,Semi-Furnished,Ground floor,North facin...",27846.153846
202,Gulshan Bellina,Noida Extension,3.0,3.0,Gulshan Homz,Greater Noida,1745.0,18,Semi-Furnished,"3 BHK Apartment for Sale in Gulshan Bellina, N...",70000000.0,70 Lac,Apartment,"3 Bath,Semi-Furnished,18 floor,East facing 3 B...",40114.613181
244,Gaur City 7th Avenue,Noida Extension,2.0,2.0,Gaurs Group,Greater Noida,1105.0,12,Semi-Furnished,2 BHK Apartment for Sale in Gaur City 7th Ave...,44200000.0,44.2 Lac,Apartment,"2 Bath,Semi-Furnished,12 floor,North facing th...",40000.000000
251,Galaxy North Avenue,Noida Extension,2.0,2.0,Galaxy Group,Greater Noida,925.0,16,Semi-Furnished,2 BHK Apartment for Sale in Galaxy North Avenu...,32000000.0,32 Lac,Apartment,"2 Bath,Semi-Furnished,16 floor,East facing Woo...",34594.594595


In [109]:
df_recommend[df_recommend.locality=='Noida Extension'].groupby(['floorno']).mean()

,bathroom,bedroom,floorSize,price,price/area
floorno,,,,,
1,2.000000,2.000000,1251.000000,3.950000e+07,29815.583984
10,2.333333,2.666667,1340.000000,4.825000e+07,35998.765681
11,2.000000,2.250000,1053.750000,4.112500e+07,39022.071512
12,1.800000,2.200000,1019.400000,3.784000e+07,37010.084088
13,2.000000,2.000000,1031.500000,4.150000e+07,39960.995034
14,2.000000,2.000000,792.000000,3.200000e+07,40404.040404
15,2.000000,2.000000,1110.000000,4.400000e+07,39639.639640
16,2.333333,2.333333,1298.000000,4.076667e+07,32060.341214
18,2.500000,2.500000,1425.000000,5.325000e+07,36573.143694


In [110]:
df_recommend[df_recommend.locality=='Noida Extension'].groupby(['bedroom']).mean()

,bathroom,floorSize,price,price/area
bedroom,,,,
1.0,1.000000,650.000000,1.770000e+07,27230.769231
2.0,1.970588,1019.088235,3.872441e+07,37806.664480
3.0,2.500000,1537.454545,5.229136e+07,33906.097572
4.0,3.000000,1885.000000,8.250000e+07,43783.120983


In [111]:
df_recommend.groupby(['locality']).mean()

,bathroom,bedroom,floorSize,price,price/area
locality,,,,,
Aimnabad,3.000000,3.000000,1210.000000,3.200000e+07,26446.280992
Alpha 1,2.166667,2.666667,1303.333333,6.200000e+07,47487.699335
Alpha 2,1.250000,1.666667,781.500000,3.775000e+07,47731.368583
Alpha 2 Block F,1.000000,NaN,480.000000,2.654400e+07,55300.000000
Amrapali Dream Valley,1.500000,1.500000,887.500000,3.400000e+07,37325.061319
"Anand Ashray, Phi 3",2.000000,2.000000,865.000000,3.250000e+07,37572.254335
Ansal Golf Links-1,5.000000,5.000000,3000.000000,9.800000e+07,32666.666667
Beta 1,3.000000,3.000000,1365.000000,5.800000e+07,42490.842491
Beta 1 Block A,2.000000,3.000000,1460.000000,4.500000e+07,30821.917808


In [113]:
df_recommend[df_recommend.locality=='Noida Extension'].groupby(['furnshingstatus']).mean()

,bathroom,bedroom,floorSize,price,price/area
furnshingstatus,,,,,
Furnished,2.333333,2.500000,1183.333333,4.190833e+07,35788.367479
Semi-Furnished,2.068966,2.206897,1154.793103,4.348207e+07,36884.959967
Unfurnished,2.192308,2.538462,1292.461538,4.510423e+07,35239.432487


In [114]:
df_recommend[df_recommend.locality=='Noida Extension'].groupby(['builderName']).mean()

,bathroom,bedroom,floorSize,price,price/area
builderName,,,,,
Ajnara India Ltd.,3.000000,3.500000,1777.500000,7.200000e+07,40178.971275
Fusion Buildtech Pvt. Ltd.,2.000000,2.500000,1390.000000,5.300000e+07,38875.790876
Galaxy Group,2.000000,2.333333,1135.000000,4.183333e+07,36640.344859
Gaurs Group,2.000000,2.000000,1101.666667,4.528333e+07,41036.514749
Gulshan Homz,3.000000,3.000000,1745.000000,7.000000e+07,40114.613181
Hawelia Group,3.000000,3.000000,1470.000000,5.145000e+07,35000.000000
JKG Group,2.000000,2.000000,1090.000000,3.488000e+07,32000.000000
Mahagun Group,2.428571,2.571429,1261.428571,5.307143e+07,41887.049723
Nirala India,2.400000,2.600000,1293.000000,4.865000e+07,37664.891149


In [117]:
df_recommend[df_recommend.locality=='Noida Extension'].groupby(['Society']).mean()

,bathroom,bedroom,floorSize,price,price/area
Society,,,,,
Ajnara Homes,3.000000,3.500000,1777.500000,7.200000e+07,40178.971275
Devika Gold Homz,2.000000,2.000000,997.000000,3.150000e+07,31594.784353
Fusion Homes,2.000000,2.500000,1390.000000,5.300000e+07,38875.790876
Galaxy North Avenue,2.000000,2.333333,1135.000000,4.183333e+07,36640.344859
Gaur City 7th Avenue,2.000000,2.000000,1127.500000,4.705000e+07,41673.819742
Gaur City 5th Avenue,2.000000,2.000000,1050.000000,4.175000e+07,39761.904762
Gulshan Bellina,3.000000,3.000000,1745.000000,7.000000e+07,40114.613181
Hawelia Valenova Park,3.000000,3.000000,1470.000000,5.145000e+07,35000.000000
JKG Palm Court,2.000000,2.000000,1090.000000,3.488000e+07,32000.000000


In [118]:
df_recommend[df_recommend.locality=='Noida Extension'].groupby(['bathroom']).mean()

,bedroom,floorSize,price,price/area
bathroom,,,,
1.0,1.250000,685.500000,2.077500e+07,29892.773893
2.0,2.250000,1118.772727,4.030977e+07,36253.199117
3.0,3.153846,1709.615385,6.372385e+07,37377.494759


In [119]:
df_recommend[df_recommend.locality=='Noida Extension'].groupby(['price']).mean()

,bathroom,bedroom,floorSize,price/area
price,,,,
17000000.0,1.000000,1.0,650.000000,26153.846154
18000000.0,1.000000,1.0,650.000000,27692.307692
18100000.0,1.000000,1.0,650.000000,27846.153846
28000000.0,2.000000,2.0,890.000000,31460.674157
29000000.0,2.000000,2.0,890.000000,32584.269663
30000000.0,1.666667,2.0,857.333333,35098.172739
31500000.0,2.000000,2.0,997.000000,31594.784353
32000000.0,2.000000,2.0,886.750000,36235.997547
33000000.0,2.000000,3.0,1365.000000,24175.824176


In [ ]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

In [ ]:
tfidf_matrix = tf.fit_transform(df_recommend['description'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
tfidf_matrix.data

In [ ]:
df_recommend.head(10).T

In [ ]:
df_recommend.Society.unique()

In [ ]:
df_recommend[df_recommend.Society=='Shri Radha Sky Garden']

In [ ]:
df_recommend.columns

In [ ]:
def citybased(city,price=None):
    df_recommend['cityName']=df_recommend['cityName'].str.lower()
    citybase=df_recommend[df_recommend['cityName']==city.lower()]
    citybase=citybase.sort_values(by='price',ascending=True)
    #print(citybase.head())
    if price is not None:
        price=int(price)
        citybase=citybase[citybase.price<=price]
    if(citybase.empty==0):
        pname=citybase[['Society', 'locality', 'bathroom', 'bedroom','floorSize', 'floorno', 'furnshingstatus', 'name', 'price',
                       'priceInWord', 'propertyType']]
        return pname.head(10)
    else:
        print('No Property Available')
   
        
            

In [ ]:
def localitybased(locality,price=None):
    df_recommend['locality']=df_recommend['locality'].str.lower()
    localitybase=df_recommend[df_recommend['locality']==locality.lower()]
    localitybase=localitybase.sort_values(by=['bedroom','bathroom','price'],ascending=False)
    #print(citybase.head())
    if price is not None:
        price=int(price)
        localitybase=localitybase[localitybase.price<=price]
    if(localitybase.empty==0):
        pname=localitybase[['Society', 'locality', 'bathroom', 'bedroom','floorSize', 'floorno', 'furnshingstatus', 'name', 'price',
                       'priceInWord', 'propertyType']]
        return pname.head(50)
    else:
        print('No Property Available')

In [ ]:
df_recommend.locality.unique()

In [ ]:
df_recommend=df_recommend[df_recommend.price!='Not define']

In [ ]:
localitybased('Noida Extention')

In [ ]:
citybased('Greater Noida')

In [ ]:
def requirementbased(city, price=None,*features):
    price=int(price)
    df_recommend['cityName']=df_recommend['cityName'].str.lower()
    #df_recommend['locality']=df_recommend['locality'].str.lower()
    features = ''.join(features)
    features=features.lower()
    features_tokens=word_tokenize(features)
    sw=stopwords.words('english')
    #description_matrix = tf.fit_transform(features)
    #cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
    lemm=WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
        #print(f_set)
    reqbased=df_recommend[df_recommend['cityName']==city.lower()]
    if price!= None:
        reqbased=reqbased[reqbased['price'] <=price]
    reqbased=reqbased.set_index(np.arange(reqbased.shape[0]))
    l1 =[];l2 =[];cos=[];
    #print(reqbased['description'])
    for i in range(reqbased.shape[0]):
        temp_tokens=word_tokenize(reqbased['description'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
            #print(temp_set)
        rvector = temp_set.intersection(f_set)
        #print(rvector)
        cos.append(len(rvector))
    reqbased['similarity']=cos
    reqbased=reqbased.sort_values(by='similarity',ascending=False)
    return reqbased[['Society', 'locality', 'bathroom', 'bedroom','floorSize', 'floorno', 'furnshingstatus', 'name', 'price',
                       'priceInWord', 'propertyType','similarity']]

In [ ]:
df_recommend.cityName.unique()

In [ ]:
requirementbased('Greater Noida' ,8000000, "3 bedroom with minimum 1400 sqft apartment in omicron 3")

In [ ]:
print(df_recommend[df_recommend.locality=='Noida Extension'].description[:1])

In [ ]:
def requirementbased(city,number,features):
    hotel['city']=hotel['city'].str.lower()
    hotel['roomamenities']=hotel['roomamenities'].str.lower()
    features=features.lower()
    features_tokens=word_tokenize(features)  
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    reqbased=hotel[hotel['city']==city.lower()]
    reqbased=reqbased[reqbased['guests_no']==number]
    reqbased=reqbased.set_index(np.arange(reqbased.shape[0]))
    l1 =[];l2 =[];cos=[];
    #print(reqbased['roomamenities'])
    for i in range(reqbased.shape[0]):
        temp_tokens=word_tokenize(reqbased['roomamenities'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        #print(rvector)
        cos.append(len(rvector))
    reqbased['similarity']=cos
    reqbased=reqbased.sort_values(by='similarity',ascending=False)
    reqbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    return reqbased[['hotelname','roomtype','guests_no','starrating','address','roomamenities','ratedescription','similarity']].head(10)

In [ ]:
def get_recommedation_budget(locality,budget=None,*features):
    #print
    df['locality']=df['locality'].str.lower()
    reqbased=df[df['locality']==locality.lower()]
    print(reqbased.head())
    if budget!=None:
        budget=int(budget)
        new_budget=budget+(budget*0.10)
        reqbased=reqbased[reqbased['price']<=(new_budget)]
    #reqbased=reqbased.set_index(np.arange(reqbased.shape[0]))
    print(get_index(reqbased,features))